# Master notebook 6/2024

In [ ]:
from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3
import sys
sys.path.append('../src')  # Adjust the path as necessary

from SPTnano import ROISelector, process_directory, generate_file_tree, display_file_tree, overlay_tracks_with_movie, plot_histograms

# from SPTnano import ROISelector, process_directory

from IPython.display import Markdown, display

import matplotlib as mpl
import matplotlib.pyplot as plt


import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience

import pims
import trackpy as tp
import os
import glob
import nd2
import seaborn as sns

In [ ]:
# Display the file tree
display_file_tree('D:\GITHUB_SOFTWARE\SPTnano')

In [ ]:
# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(10, 5))
mpl.rc('image', cmap='gray')

sns.set_context("notebook", rc={"xtick.labelsize": 10, "ytick.labelsize": 10})

In [ ]:
nd2files = 'D:/6-25-2024_trackercomparison/ND2_test_data/'
master = 'D:/6-25-2024_trackercomparison/raw_data_notinverted/'
saved_data_dir = 'D:/6-25-2024_trackercomparison/TrackPy_notinverted_Ultra100ms/'

# Use glob to find TIF files in the directory and its subdirectories
tif_files = glob.glob(saved_data_dir + '**/*.tif', recursive=True)
# Print the list of TIF files
print(tif_files)

OPTIONAL: crop ND2 images and make into tiffs

In [ ]:


# Set the ROI size
roi_width = 150
roi_height = 150

# Run the function to process the directory
# batch_roi_selector.process_directory(nd2_directory, tif_directory, roi_width, roi_height) #
process_directory(nd2files, master, roi_width, roi_height) #


Main pytrack pipeline:

In [ ]:
frames = pims.open(tif_files[0])

In [ ]:
detection_circle_size = 7 # in SLIMfast this was 13 for 10 ms
f = tp.locate(frames[0], detection_circle_size, invert=False)
tp.annotate(f, frames[0]);

In [ ]:
fig, ax = plt.subplots()
ax.hist(f['mass'], bins=20)

# Optionally, label the axes.
ax.set(xlabel='mass', ylabel='count');

In [ ]:
f = tp.locate(frames[0], 7, invert=False, minmass=150)
tp.annotate(f, frames[0])

In [ ]:
tp.subpx_bias(tp.locate(frames[2], 7, invert=False, minmass=150))

In [ ]:
f = tp.batch(frames[:], 7, minmass=150, invert=False, engine='numba')

In [ ]:
# tp.quiet()  # Turn off progress reports for best performance
linking_max_distance = 10
disappearance_max_frames = 0
t = tp.link(f, linking_max_distance, memory=disappearance_max_frames) 

In [ ]:
print(len(t.particle.unique()))

In [ ]:
t1 = tp.filter_stubs(t, 2) # for 100 ms data, 2 frames is 0.2 seconds. In tiago's SLIMfast data, this was 5 frames at 10 ms, which is 50 ms. As a track is not a track unless it has 2 frames, this is the minimal value we can put.
minimum_track_length = 2
# Compare the number of particles in the unfiltered and filtered data.
print('Before:', t['particle'].nunique())
print('After:', t1['particle'].nunique())

In [ ]:
plt.figure()
tp.mass_size(t1.groupby('particle').mean()); # convenience function -- just plots size vs. mass

In [ ]:
plt.figure()
tp.annotate(t[t['frame'] == 0], frames[0])

In [ ]:
plt.figure()
tp.annotate(t1[t1['frame'] == 0], frames[0])

In [ ]:
# plot the ecc as a histogram
plt.figure()
t1['ecc'].hist(bins=100)
plt.xlabel('eccentricity')
plt.ylabel('count')


In [ ]:
plt.figure()
tp.annotate(t2[t2['frame'] == 0], frames[0]);

BATCH TRACKPY

In [ ]:
####### detection settings ######## FOR 100 ms!!! #######

detection_circle_size = 7 # set for 100 ms
minimum_intensity = 150 # set for 100 ms
iterations = 50
svn_threshold = 10
noise_size = 1.01017
engine = 'numba'

####### linking settings ########
linking_max_distance = 7
disappearance_max_frames = 2

####### filtering settings ########
minimum_track_length = 5

size = 2

# Create an empty master dataframe
master_df = pd.DataFrame()

# Drift correction
drift_correction = False
# filtering needed?
filtering = False

tp.quiet()  # Turn off progress reports for best performance
for i in range(len(tif_files)):
    frames = pims.open(tif_files[i])
    f = tp.batch(frames[:], detection_circle_size, minmass=minimum_intensity, invert=True, max_iterations=iterations, threshold=svn_threshold)
    t = tp.link(f, linking_max_distance, memory=disappearance_max_frames)
    t1 = tp.filter_stubs(t, minimum_track_length)
    print('Before:', t['particle'].nunique())
    print('After:', t1['particle'].nunique())
    if filtering == True:
        t2 = t1[((t1['mass'] > minimum_intensity) & (t1['size'] < size))]
    else:
        t2 = t1.copy()
    if drift_correction == True:
        d = tp.compute_drift(t2)
        tm = tp.subtract_drift(t2.copy(), d)
    else:
        tm=t2.copy()
    # Add column with the filename
    tm['filename'] = os.path.splitext(os.path.basename(tif_files[i]))[0]
    # Append tm dataframe to the master dataframe
    master_df = pd.concat([master_df, tm])

    # tm.to_csv(tif_files[i] + '.csv')
    print(tif_files[i] + ' processed')
    print('-------------------------------------')
    # reset index
master_df = master_df.reset_index(drop=True)
# create a file ID (integer) based on each filename
master_df['file_id'] = pd.Categorical(master_df['filename']).codes
# # create a uniq_id for each particle, based on the filename and the particle number
master_df['unique_id'] = master_df['file_id'].astype(str) + '_' + master_df['particle'].astype(str)

master_df.to_csv(saved_data_dir + 'master_df.csv')

In [ ]:
# read in the master_df
master_df = pd.read_csv(saved_data_dir + 'master_df.csv')

In [ ]:
master_df

In [ ]:
# select a random number from the unique particles
random_particle = np.random.choice(master_df['particle'].unique())

# extract a new 'particle df' for a random particle


particle_df = master_df[master_df['particle'] == random_particle]


In [ ]:
# Count the number of frames for each particle
frame_counts = master_df['unique_id'].value_counts()

# Compute the mean track length across all particles
mean_track_length_value_counts = frame_counts.mean()
median_track_length_value_counts = frame_counts.median()

print('Mean track length: ' + str(mean_track_length_value_counts))
print('Median track length: ' + str(median_track_length_value_counts))


In [ ]:
SLIMfast_df

In [ ]:
plt.figure(figsize=(6, 6))  # Set the figure size to create a square plot
# extract a df for each filename
filenames = SLIMfast_df['filename'].unique()
print(filenames)
picked_filename = filenames[3]
df = SLIMfast_df[SLIMfast_df['filename'] == picked_filename]


f = tp.plot_traj(df)
# add title to f
f.set_title(picked_filename)

Movie function. Takes the current tracks_df and the full movie path you want to convert

This gets the tracks_df and movie path. Input is the name of the file you want to make a movie for

In [ ]:
print(f'Possible filenames are: {master_df["filename"].unique()}')

filename = 'RUES2_kinesinhaloJFX554_laser25_009_cropped'
tracks_df = master_df[master_df['filename'] == filename]
tracks_df.reset_index(drop=True, inplace=True)

directory = os.path.dirname(tif_files[0])
# print(directory)
# 
movie_path = os.path.join(directory, filename + '.tif')    

In [ ]:
SLIMfast_df

In [ ]:
print(f'Possible filenames are: {SLIMfast_df["filename"].unique()}')

filename = 'RUES2_kinesinhaloJFX554_laser25_009_cropped_tracked'
tracks_df = SLIMfast_df[SLIMfast_df['filename'] == filename]
tracks_df.reset_index(drop=True, inplace=True)

directory = os.path.dirname(SLIMfastdatapath[0])
print(directory)
print(SLIMfastdatapath)
# # 
# movie_path = os.path.join(SLIMfastdatapath, filename + '.tif')    
# print(movie_path)

In [ ]:
overlay_tracks_with_movie(tracks_df, movie_path, colormap=None)

# SLIMfast data upload

In [ ]:
SLIMfastdatapath='D:/6-25-2024_trackercomparison/SLIMfast_notinverted_Ultra100ms/Condition_100msUltra_matchedwithtrackpy_1poin2searchfactor_trackedexport/'

In [ ]:
SLIMfastdatapath='D:/6-25-2024_trackercomparison/SLIMfast_notinverted_Ultra100ms/Condition_100msUltra_matchedwithtrackpy_1poin2searchfactor_trackedexport/'
# list files
files = os.listdir(SLIMfastdatapath)
SLIMfast_df = pd.DataFrame()
Colnames = ['x', 'y', 'frame', 'particle', 'factor1', 'factor2', 'factor3', 'factor4',]

for file in files:
    # print(file)
    df = pd.read_csv(SLIMfastdatapath + file, sep='\t', names=Colnames)

    df['filename'] = os.path.splitext(os.path.basename(file))[0]
# Append tm dataframe to the master dataframe
    SLIMfast_df = pd.concat([SLIMfast_df, df])


SLIMfast_df = SLIMfast_df.reset_index(drop=True)
# create a file ID (integer) based on each filename
SLIMfast_df['file_id'] = pd.Categorical(SLIMfast_df['filename']).codes
# # create a uniq_id for each particle, based on the filename and the particle number
SLIMfast_df['unique_id'] = SLIMfast_df['file_id'].astype(str) + '_' + SLIMfast_df['particle'].astype(str)





In [ ]:
SLIMfast_df.to_csv(SLIMfastdatapath + 'SLIMfast_df.csv', index=False)

In [ ]:
# load both dfs
master_df = pd.read_csv(saved_data_dir + 'master_df.csv')
SLIMfast_df = pd.read_csv(SLIMfastdatapath + 'SLIMfast_df.csv')

In [ ]:
SLIMfast_df

In [ ]:
combined_df

In [ ]:
## Glorious - you now have a SLIMfast_df and a master_df == trackpy_df. You could easily make a histogram.

# The movies from here - can you check what the matlab code does, in order to cut off the tracks that have disappeared? 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


# Assuming 'master_df' has a 'filename' column and 'unique_id' for track lengths
plt.figure(figsize=(10, 6))

# Loop through each unique filename and plot its histogram
for i, filename in enumerate(master_df['filename'].unique()):
    subset = master_df[master_df['filename'] == filename]
    file_id = subset['file_id'].unique()
    # add 1 to file_id to make it human readable and remove the []
    file_id = file_id[0] + 1

    shift = i * 0.05
    sns.histplot(subset['unique_id'].value_counts(), kde=True, bins=100, label=filename, alpha=0.5)
    subset_mean = subset['unique_id'].value_counts().mean()
    subset_median = subset['unique_id'].value_counts().median()
    subset_number_of_tracks = len(subset['unique_id'].unique())
    plt.text(0.5, 0.7-shift, f"{file_id}: mean: {subset_mean:.2f} frames from {subset_number_of_tracks} tracks", transform=plt.gca().transAxes)

plt.xlabel('Track length (frames)')
plt.ylabel('Count')
plt.legend(title='Filename')
plt.title('Overlaid Histograms by Filename')
plt.show()

In [ ]:
TrackPy_df = master_df

In [ ]:
#### DATAFRAME PRUNING AND CONCATENATION ######

# Add a column that just says trackpy in every row of the TrackPy_df
TrackPy_df['tracker'] = 'TrackPy'
# same with slimfast
SLIMfast_df['tracker'] = 'SLIMfast'
# concatenate the two dataframes

# get columns names of the two dataframes
print(TrackPy_df.columns)
print(SLIMfast_df.columns)

# remove all of the columns apart from x, y, frame, particle, filename, unique_id, tracker, file id
TrackPy_df_sub = TrackPy_df[['x', 'y', 'frame', 'particle', 'filename', 'unique_id', 'tracker', 'file_id']]
SLIMfast_df_sub = SLIMfast_df[['x', 'y', 'frame', 'particle', 'filename', 'unique_id', 'tracker', 'file_id']]

# concatenate the two dataframes
combined_df = pd.concat([TrackPy_df_sub, SLIMfast_df_sub])

# save it
combined_df.to_csv(saved_data_dir + 'combined_df.csv', index=False)


In [ ]:


framerate = 0.1 #seconds per frame
# value counts of the unique_id
valuecountstrackpy = TrackPy_df['unique_id'].value_counts()
valuecountsslimfast = SLIMfast_df['unique_id'].value_counts()
# print(valuecounts)
# translate each value count to time in seconds by multiplying by 0.1
valuecounts_secondstrackpy = valuecountstrackpy * framerate
valuecounts_secondsslimfast = valuecountsslimfast * framerate
# valuecounts_seconds

sns.histplot(valuecounts_seconds, kde=True, bins = 100)
plt.xlabel('Track length (seconds)')
plt.ylabel('Count')

# Annotate plot with mean and median
mean = valuecounts_seconds.mean()
median = valuecounts_seconds.median()
plt.text(0.5, 0.9, f"Mean: {mean:.2f} seconds", transform=plt.gca().transAxes)
plt.text(0.5, 0.85, f"Median: {median:.2f} seconds", transform=plt.gca().transAxes)
plt.text(0.5, 0.8, f"Number of tracks: {len(master_df['unique_id'].unique())}", transform=plt.gca().transAxes)

plt.show()
plt.show()


In [ ]:
framerate = 0.1 #seconds per frame

# Assuming 'master_df' has a 'filename' column and 'unique_id' for track lengths
plt.figure(figsize=(20, 12))
size=10
multiplier = 2
sns.set_context("notebook", rc={"xtick.labelsize": size*multiplier, "ytick.labelsize": size*multiplier})

# Loop through each unique filename and plot its histogram
for i, tracker in enumerate(combined_df['tracker'].unique()):
    subset = combined_df[combined_df['tracker'] == tracker]
    subsetvalues = subset['unique_id'].value_counts()
    subsetvalues_seconds = subsetvalues * framerate
    # make these into a percentage of the total number of tracks


    shift = i * 0.05
    sns.histplot(subsetvalues_seconds, kde=True, stat='percent', bins=100, label=tracker, alpha=0.5, )
    subset_mean = subsetvalues_seconds.mean()
    subset_median = subsetvalues_seconds.median()
    subset_number_of_tracks = len(subset['unique_id'].unique())
    plt.text(0.55, 0.8-shift, f"{tracker}: mean: {subset_mean:.2f} seconds from {subset_number_of_tracks} tracks", transform=plt.gca().transAxes, fontsize=10*multiplier)

plt.xlabel('Track length (seconds)', fontsize=size*multiplier)
plt.ylabel('Percentage', fontsize=size*multiplier)
plt.legend(title='', fontsize=size*multiplier)
# Get the current axis
ax = plt.gca()

# Set the x-axis and y-axis limits
# ax.set_xlim(0, 10)  # Set the x-axis limits
# ax.set_ylim(0, 30)   # Set the y-axis limits
# plt.title('Overlaid Histograms by Tracker')
plt.show()

# increase the resolution ofe the plot
# plt.figure(figsize=(10, 6))


In [ ]:
# Does trackpy have tracks shorter than 5 frames?
# Does SLIMfast have tracks shorter than 5 frames?
# calculate
# value counts of the unique_id
valuecountstrackpy = TrackPy_df['unique_id'].value_counts()
valuecountsslimfast = SLIMfast_df['unique_id'].value_counts()
print(min(valuecountstrackpy))
print(min(valuecountsslimfast))

In [ ]:
plt.figure(figsize=(6, 6))  # Set the figure size to create a square plot
# extract a df for each filename
filenames = master_df['filename'].unique()
print(filenames)
picked_filename = filenames[3]
df = master_df[master_df['filename'] == picked_filename]


f = tp.plot_traj(df)
# add title to f
f.set_title(picked_filename)


# plt.axis('equal')  # Set the aspect ratio to be equal
# make the title the filename
# plt.title(filenames[0])
# plt.show()

Function to replace tps plotting one:

In [ ]:
plot_histograms(combined_df, framerate=0.1, bins=200)